In [1]:
!pip install --upgrade pip einops

In [2]:
import pandas as pd
from transformers import pipeline

In [3]:
# List of Tagalog sentences
data = {
    "Sentence": [
        "Magandang umaga sa inyong la",
        "Ako ay masayang makita ka mu",
        "Pupunta ako sa palengke m",
        "Mahal kita ng buong pu",
        "Kumain tayo ng adobo para sa hap",
        "Sana ay maging matagumpay ang iyong ara",
        "Nag-aaral ako ng programming ar",
        "Gusto kong magbakasyon sa Bagu",
        "Ang ganda ng tanawin sa probin",
        "Mayroon akong alagang aso at pu"
    ]
}

# Create DataFrame
original_df = pd.DataFrame(data)
print(original_df)

                                  Sentence
0             Magandang umaga sa inyong la
1             Ako ay masayang makita ka mu
2                Pupunta ako sa palengke m
3                   Mahal kita ng buong pu
4         Kumain tayo ng adobo para sa hap
5  Sana ay maging matagumpay ang iyong ara
6          Nag-aaral ako ng programming ar
7           Gusto kong magbakasyon sa Bagu
8           Ang ganda ng tanawin sa probin
9          Mayroon akong alagang aso at pu


In [4]:
# Function to predict exactly 3 words
def predict_top_three(pipe, typed_characters):
    sentence = f"{typed_characters}<mask>"
    predictions = pipe(sentence, top_k=3)
    return [pred['token_str'] for pred in predictions]

# Tagalog Base

In [5]:
# Load the fill-mask pipeline with the specified model
tagalog_base = pipeline("fill-mask", model="jcblaise/roberta-tagalog-base")

In [6]:
tagalog_base_df = original_df.copy()
# Apply the prediction function and split the results into new columns in one line
tagalog_base_df[['Word_1', 'Word_2', 'Word_3']] = pd.DataFrame(
    tagalog_base_df['Sentence'].apply(lambda x: predict_top_three(tagalog_base, x)).tolist(), 
    index=tagalog_base_df.index
)
tagalog_base_df

,Sentence,Word_1,Word_2,Word_3
0,Magandang umaga sa inyong la,...,.,!
1,Ako ay masayang makita ka mu,.,!,...
2,Pupunta ako sa palengke m,...,useum,.
3,Mahal kita ng buong pu,...,erto,.
4,Kumain tayo ng adobo para sa hap,.,...,los
5,Sana ay maging matagumpay ang iyong ara,.,!,...
6,Nag-aaral ako ng programming ar,te,ena,cade
7,Gusto kong magbakasyon sa Bagu,ilat,ia,zon
8,Ang ganda ng tanawin sa probin,.,...,olohiya
9,Mayroon akong alagang aso at pu,...,…,erto


# Tagalog Large

In [7]:
# Load the fill-mask pipeline with the specified model
tagalog_large = pipeline("fill-mask", model="jcblaise/roberta-tagalog-large")

In [8]:
tagalog_large_df = original_df.copy()
# Apply the prediction function and split the results into new columns in one line
tagalog_large_df[['Word_1', 'Word_2', 'Word_3']] = pd.DataFrame(
    tagalog_large_df['Sentence'].apply(lambda x: predict_top_three(tagalog_large, x)).tolist(), 
    index=tagalog_large_df.index
)
tagalog_large_df

,Sentence,Word_1,Word_2,Word_3
0,Magandang umaga sa inyong la,...,.,!
1,Ako ay masayang makita ka mu,.,!,...
2,Pupunta ako sa palengke m,...,.,onday
3,Mahal kita ng buong pu,...,.,!
4,Kumain tayo ng adobo para sa hap,.,...,!
5,Sana ay maging matagumpay ang iyong ara,...,.,!
6,Nag-aaral ako ng programming ar,ray,te,ena
7,Gusto kong magbakasyon sa Bagu,ilat,quin,.
8,Ang ganda ng tanawin sa probin,.,...,!
9,Mayroon akong alagang aso at pu,...,.,erto


In [9]:
aisingapore_sealion_bert_base = pipeline("fill-mask", model="aisingapore/sealion-bert-base", trust_remote_code=True)

/home/zeus/.cache/huggingface/modules/transformers_modules/aisingapore/sealion-bert-base/1c74a03d55693066c66b67a3ab0f0bf9193d5ccd/bert_layers.py:178: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


In [10]:
# Function to predict exactly 3 words
def predict_top_three(pipe, typed_characters):
    sentence = f"{typed_characters}<|mask|>"
    predictions = pipe(sentence, top_k=3)
    return [pred['token_str'] for pred in predictions]

In [11]:
aisingapore_sealion_bert_base_df = original_df.copy()
# Apply the prediction function and split the results into new columns in one line
aisingapore_sealion_bert_base_df[['Word_1', 'Word_2', 'Word_3']] = pd.DataFrame(
    aisingapore_sealion_bert_base_df['Sentence'].apply(lambda x: predict_top_three(aisingapore_sealion_bert_base, x)).tolist(), 
    index=aisingapore_sealion_bert_base_df.index
)
aisingapore_sealion_bert_base_df

,Sentence,Word_1,Word_2,Word_3
0,Magandang umaga sa inyong la,!,.,...
1,Ako ay masayang makita ka mu,.,!,...
2,Pupunta ako sa palengke m,.,...,ami
3,Mahal kita ng buong pu,westo,wede,wers
4,Kumain tayo ng adobo para sa hap,unan,...,on
5,Sana ay maging matagumpay ang iyong ara,.,on,in
6,Nag-aaral ako ng programming ar,al,mas,alin
7,Gusto kong magbakasyon sa Bagu,.,!,?
8,Ang ganda ng tanawin sa probin,ya,iya,aya
9,Mayroon akong alagang aso at pu,.,hod,...


# GPT2 tagalog

In [12]:
# Load the fill-mask pipeline with the specified model
gpt2 = pipeline("text-generation", model="jcblaise/gpt2-tagalog")

In [17]:
# Function to predict the next word based on typed characters
def suggest_next_words(typed_characters, num_sequences=3):
    # Generate text with specified sampling parameters
    output = gpt2(
        typed_characters,
        max_length=len(typed_characters.split()) + 5,  # One extra word
        num_return_sequences=num_sequences,
        do_sample=True,  
        top_k=50,
        top_p=0.95
    )
    
    # Extract the generated sentences and return the last word
    next_words = [result['generated_text'].strip() for result in output]
    
    return next_words

In [18]:
gpt2_df = original_df.copy()
# Apply the function and split the results into new columns in one line
gpt2_df[['Word_1', 'Word_2', 'Word_3']] = pd.DataFrame(
    gpt2_df['Sentence'].apply(lambda x: suggest_next_words(x)).tolist(), 
    index=gpt2_df.index
)

# Display the DataFrame with predictions
gpt2_df

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


,Sentence,Word_1,Word_2,Word_3
0,Magandang umaga sa inyong la,Magandang umaga sa inyong laundry shop para,Magandang umaga sa inyong laundry visit sa,Magandang umaga sa inyong laundry greet
1,Ako ay masayang makita ka mu,Ako ay masayang makita ka mu man.B: Kum,Ako ay masayang makita ka mu**aaaa,Ako ay masayang makita ka mu makapausap ka ni
2,Pupunta ako sa palengke m,Pupunta ako sa palengke maming pamilya 'yan,Pupunta ako sa palengke manantaw ug,Pupunta ako sa palengke mada-ling ko
3,Mahal kita ng buong pu,Mahal kita ng buong pueblo ay tinitignan,Mahal kita ng buong puesto ang napunta kay,Mahal kita ng buong puede sa isang taon
4,Kumain tayo ng adobo para sa hap,Kumain tayo ng adobo para sa haplit at walang,Kumain tayo ng adobo para sa haplit ng isda,Kumain tayo ng adobo para sa hapong prutas gaya
5,Sana ay maging matagumpay ang iyong ara,Sana ay maging matagumpay ang iyong arauche pe...,Sana ay maging matagumpay ang iyong arauchemen...,Sana ay maging matagumpay ang iyong arauchetics.
6,Nag-aaral ako ng programming ar,Nag-aaral ako ng programming aria para,Nag-aaral ako ng programming arnis.,Nag-aaral ako ng programming arrows
7,Gusto kong magbakasyon sa Bagu,"Gusto kong magbakasyon sa Baguioka, Japan nang",Gusto kong magbakasyon sa Baguioka para makauw...,"Gusto kong magbakasyon sa Bagui, sa lahat,"
8,Ang ganda ng tanawin sa probin,Ang ganda ng tanawin sa probinion ni Spiel,Ang ganda ng tanawin sa probinido ( o s,Ang ganda ng tanawin sa probinema na iyon ng
9,Mayroon akong alagang aso at pu,Mayroon akong alagang aso at puffer para sa pag-,Mayroon akong alagang aso at pufferan sa mga a...,"Mayroon akong alagang aso at puffer dogs, kaya"


In [ ]:
sea_llm_v3_7b_chat = pipeline("text-generation", model="SeaLLMs/SeaLLMs-v3-7B-Chat")